In [11]:
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix 

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-03-18 06:28:21.815936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 06:28:21.967451: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glob/development-tools/versions/oneapi/2023.0.1/oneapi/vpl/2023.0.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/tbb/2021.8.0/env/../lib/intel64/gcc4.8:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/rkcommon/1.10.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/ospray_studio/0.11.1/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/ospray/2

In [5]:
# loading the model densenet
model_1 = tf.keras.models.load_model('models/densenet.h5')

In [6]:
model_2 = tf.keras.models.load_model('models/resnet.h5')

In [12]:
# Load the test CSV file with filenames
t_test_df = pd.read_csv("Emergency_Vehicles/train.csv")

# Create a generator for the test images
t_test_datagen = ImageDataGenerator(rescale=1./255)
t_test_generator = t_test_datagen.flow_from_dataframe(
    t_test_df,
    directory="Emergency_Vehicles/train",
    x_col='image_names',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False)

Found 1380 validated image filenames.


/home/u184510/.local/lib/python3.9/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 266 invalid image filename(s) in x_col="image_names". These filename(s) will be ignored.
  warnings.warn(


In [15]:
# Use the trained model to make predictions on the test data
preds_1 = model_1.predict(t_test_generator, steps=None)
preds_2 = model_2.predict(t_test_generator, steps=None)

44/44 [==============================] - 32s 728ms/step


In [16]:
# Remove any missing filenames from the test CSV file
missing_filenames = set(t_test_generator.filenames) - set(t_test_df['image_names'])
if missing_filenames:
    print(f'Removing {len(missing_filenames)} missing filenames from the test CSV file.')
    t_test_df = t_test_df[~t_test_df['image_names'].isin(missing_filenames)]

In [25]:
#Ensembling the two models
li_ense = (preds_1.ravel()*0.8)+(preds_1.ravel()*0.2)
li = []
for i in li_ense:
    if i>0.5:
        li.append(1)
    else:
        li.append(0)

In [26]:
# Create a DataFrame with the filenames and predictions
df = pd.DataFrame({'image_names': t_test_generator.filenames[:len(preds_1)], 'emergency_or_not': li})

test_inst_df=pd.merge(df, t_test_df, on='image_names', how='inner')

In [27]:
accuracy_score(df["emergency_or_not"],test_inst_df['emergency_or_not_y']) 

0.9818840579710145

In [28]:
confusion_matrix(df["emergency_or_not"],test_inst_df['emergency_or_not_y']) 

array([[723,  24],
       [  1, 632]])